In [8]:
import numpy as np
from math import sin, cos, sqrt, atan2, radians
import pandas as pd

1. Командная строка: в поиске  cmd
2. Посмотреть содержимое: dir
3. Комадна для смены дерриктории: cd /название_папки
4. Для того чтобы подняться вверх по каталогу: cd ..
5. Для включения питона набрать: jupyter notebook

In [2]:
a = np.array([[1,2,3],
              [1,1,1],
              [0,0,1]])

In [3]:
print("Матрица А:\n",a)
print("Матрица A транспонированная:\n",a.T)
print("Матрица обратная A:\n", np.linalg.inv(a))
print("Матрица A^2:\n", np.dot(a,a))

Матрица А:
 [[1 2 3]
 [1 1 1]
 [0 0 1]]
Матрица A транспонированная:
 [[1 1 0]
 [2 1 0]
 [3 1 1]]
Матрица обратная A:
 [[-1.  2.  1.]
 [ 1. -1. -2.]
 [ 0.  0.  1.]]
Матрица A^2:
 [[3 4 8]
 [2 3 5]
 [0 0 1]]


In [6]:
def distance_between_points(lat_a, lon_a, lat_b, lon_b):

    R = 6373.0

    lat1 = radians(lat_a)
    lon1 = radians(lon_a)
    lat2 = radians(lat_b)
    lon2 = radians(lon_b)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [16]:
df = pd.read_excel('Координаты.xlsx')

In [18]:
df.head()

,RegionNameORIGIN,Origin Latitude,Origin Longitude
0,Республика Башкортостан,54.734856,55.957856
1,Республика Бурятия,51.833507,107.584125
2,Республика Дагестан,42.983060,47.504682
3,Кабардино-Балкарская республика,43.484605,43.607090
4,Республика Калмыкия,46.308309,44.270181


In [20]:
#
lat_lon = df[['Origin Latitude','Origin Longitude']].values

In [23]:
matr_dist = np.zeros((79,79,79))

# Выбор h
1. $AIC = 2k+n[ln(2\pi RSS/n) +1]$
2. $RSS = \sum_{i=1}^{N}(y_i-h)^2$

In [26]:
for i in range(79):
    for j in range(79):
        matr_dist[i,j,j] = distance_between_points(lat_lon[i,0],lat_lon[i,1], 
                                                   lat_lon[j,0],lat_lon[j,1])

In [79]:
h = 0
for i in range(79):
    for j in range(79):
        h += (distance_between_points(lat_lon[i,0],lat_lon[i,1], 
                                 lat_lon[j,0],lat_lon[j,1])-2419.54)**2/79/79

In [81]:
h = h**(0.5)

In [82]:
h

1968.5992536211638

In [84]:
W = np.zeros((79,79,79))
for i in range(79):
    for j in range(79):
        W[i,j,j] = np.exp((-0.5)*(distance_between_points(lat_lon[i,0],lat_lon[i,1], 
                                                   lat_lon[j,0],lat_lon[j,1])/h)**2)

In [28]:
x_df = pd.read_excel('данные 2002 и 2016 год.xlsx')

In [52]:
x_df_2002 = x_df[['TFR','Gender gap', 'Secularisation', 'GDP per inhabitant']]
x_df_2016 = x_df[['Unnamed: 1','Unnamed: 3', 'Unnamed: 5', 'Unnamed: 7']]

In [53]:
x_df_2002['GDP sqare'] = x_df_2002['GDP per inhabitant']*x_df_2002['GDP per inhabitant']

/home/artyom/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [54]:
x_df_2002 = x_df_2002[1:]

In [55]:
x_df_2016['GDP sqare'] = x_df_2016['Unnamed: 7']*x_df_2016['Unnamed: 7']

/home/artyom/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [56]:
x_df_2016 = x_df_2016[1:]

In [58]:
x_df_2002 = x_df_2002 - x_df_2002.mean()
x_df_2016 = x_df_2016 - x_df_2016.mean()

In [59]:
x_df_2002 = x_df_2002/x_df_2002.std()
x_df_2016 = x_df_2016/x_df_2016.std()

In [61]:
x_df_2002['const'] = 1
x_df_2016['const'] = 1

In [113]:
X_2002 = x_df_2002[['Gender gap', 'Secularisation', 'GDP per inhabitant','GDP sqare','const']].values
X_2016 = x_df_2016[['Unnamed: 3', 'Unnamed: 5', 'Unnamed: 7','GDP sqare','const']].values

In [114]:
Y_2002 = x_df_2002.TFR.values
Y_2016 = x_df_2016['Unnamed: 1'].values

In [115]:
out_df_2002 = pd.DataFrame()
for i in range(79):
    inv_matr = np.linalg.inv(np.dot(np.dot(X_2002.T,W[i]),X_2002))
    beta_i = np.dot(np.dot(np.dot(inv_matr,X_2002.T),W[i]),Y_2002)
    _answer = dict(zip(['Gender gap', 'Secularisation', 'GDP per inhabitant','GDP sqare','const'],beta_i))
    out_df_2002 =  out_df_2002.append(_answer,ignore_index=True)

In [124]:
out_df_2002 = out_df_2002.set_index(x_df_2002.index)

In [125]:
out_df_2002.to_excel('coef_2002.xlsx')

In [117]:
out_df_2016 = pd.DataFrame()
for i in range(79):
    inv_matr = np.linalg.inv(np.dot(np.dot(X_2016.T,W[i]),X_2016))
    beta_i = np.dot(np.dot(np.dot(inv_matr,X_2016.T),W[i]),Y_2016)
    _answer = dict(zip(['Gender gap', 'Secularisation', 'GDP per inhabitant','GDP sqare','const'],beta_i))
    out_df_2016 = out_df_2016.append(_answer,ignore_index=True)

In [126]:
out_df_2016 = out_df_2016.set_index(x_df_2016.index)

In [127]:
out_df_2016.to_excel('coef_2016.xlsx')